In [1]:
import torch 
import torch.nn as nn 
import numpy as np 
from torch.utils.data import Dataset
import glob
import cv2
import os 
from torch.autograd import Variable

In [2]:
model_path = "/home/rajat/rajat/fastai/dogscats/sample/train/cats/"


### Custom Data Loader 

In [3]:
class data_loader(Dataset):
    def __init__(self, path, transform=False, file_suffix = "*.jpg", label = None, batch_size = 32):
        super().__init__()
        self.path = path
        self.files = glob.glob(os.path.join(path ,"*.jpg"))
        self.transform = transform
        self.label = torch.cuda.LongTensor(label)
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = self.files[idx]
        img = cv2.imread(img_name)
        img = cv2.resize(img, dsize = (200,200))
        img = np.einsum('ijk->kij', img)
        img = np.expand_dims(img, axis =0)
        img = torch.from_numpy(img)
#         img = torch.FloatTensor(img)
        d = {'image'  : img, 'name' : img_name, 'label' : self.label }
        return d
        

In [4]:
loader = data_loader(path = model_path, label=1)

### First model 

In [5]:
def return_conv_block(input_size, output_size, stride_conv=1, stride_pool=2, kernel_size_conv=3, kernel_size_pool=2):
    return nn.Sequential(
        nn.Conv2d(input_size, output_size, kernel_size=(kernel_size_conv, kernel_size_conv), stride=stride_conv),
        nn.BatchNorm2d(output_size),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=kernel_size_pool, stride=2 )
    )
        
    

class CNN(nn.Module):
    """
        This implements basic cnn module
    """
    def __init__(self, num_classes = 2):
        super().__init__()
        self.conv1 = return_conv_block(input_size=3, output_size=20)
        self.conv2 = return_conv_block(input_size=20, output_size=40)
        self.conv3 = return_conv_block(input_size=40, output_size=80)
        self.fc = nn.Linear(23 * 23 * 80, num_classes)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.view(out.size(0), -1) # Flatten the output for linear layer 
        return self.fc(out)
    
cnn = CNN()
cnn.cuda()

CNN (
  (conv1): Sequential (
    (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (conv2): Sequential (
    (0): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (conv3): Sequential (
    (0): Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (fc): Linear (42320 -> 2)
)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr = 0.001)

In [7]:
loss = []
num_epoch = 2
for epoch in range(num_epoch):
    for k, d in enumerate(loader):
        img = d['image']
        label = d['label']
        img = Variable(img).cuda()
        label = Variable(label).cuda()
        # optimize initialize zero grad 
        optimizer.zero_grad()
        output = cnn(img.float())
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()


torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size([1, 80, 23, 23])
torch.Size([1, 42320])
torch.Size([1, 3, 200, 200])
torch.Size(

In [8]:
img = np.arange(20).reshape(2,5,2)
a = np.expand_dims(img, axis =0)

In [9]:
a.shape

(1, 2, 5, 2)